### Laboratorio de Escalonamento ###

Neste laboratório, iremos simular o funcionamento de algoritmos de escalonamento básicos para entender melhor seu funcionamento.

Na célula abaixo, temos uma classe Processo, que tem as informações de execução, e uma classe de exemplo de escalonamento apenas com os protótipos:

(orientação a objeto em Python)

```python
import numpy as np
import random

class Processo(object):
    def __init__(self,pnome,pio,ptam,prioridade,tempoChegada,bilhetes=0):
        self.nome = pnome
        self.io = pio # Probabilidade de fazer E/S, inicialmente zero
        self.tam = ptam # Quantos Timeslices sao necessarios para terminar
        self.prio = prioridade # Prioridade, eh desnecessaria agora 
        self.chegada = 0
        
    def roda(self,quantum=None): # se rodar sem quantum, o processor roda ate o fim
        if(random.randint(1,100)<self.io): #Verifica se fez E/S
            self.tam-=1
            print(self.nome," fez e/s, falta ",self.tam)
            return 1, True #True que fez E/S
            
            
        if(quantum is None or self.tam<quantum):
            quantum = self.tam
        self.tam -=quantum
        print(self.nome," rodou por ",quantum," timeslice, faltam ",self.tam)
        return quantum, False # False se nao fez E/S
    

class escalonador(object): # Protótipo de escalonador de exemplo
    def __init__(self,vprontos=[]):
        self.prontos = vprontos #processos que cheam ao tempo zero

    def pronto(self,Processo):
        # implemente aqui o que escalonador faz quando surge um novo processo pronto
        
    def proximo(self):
        # implemente aqui a politica de escalonamento para definir um processo a ser executado
        return p #processo p eh escolhido

Na célula abaixo, são criados quatro processos completamente CPU-Bound que precisam de 3 timeslices para rodar.

O valor de E/S é um número entre 0 e 100 indicando quantos porcento de chance o processo tem de fazer E/S durante seu tempo na CPU

```python
nprocs = 4
nomes = ['A','B','C','D']
chanceio = [0,0,0,0] #Valor de zero a cem, chance de ser entrada e saida por enquanto deixem em zero
tamanho = np.array([3,3,3,3])


total = tamanho.sum()

procs = []
for i in range(nprocs):
    procs.append(Processo(nomes[i],chanceio[i],tamanho[i],0,0)) #cria uma lista procs de Processos

Na célula abaixo, temos o mesmo simulador do laboratório anterior:

```python

quantum = 2
tempoBloq = 1

escalonador = escalonador(procs) #troque escalonador pelo seu escalonador
bloqueados = []

tempo = 0

random.seed(0)

while total>0:
    p = escalonador.proximo()
    if(p is not None):
        rodou, _ = p.roda() #adicione quantum como parâmetro, por enquanto nao temos E/S
        
        if(p.tam>0):
            escalonador.pronto(p)
        total-=rodou
        tempo+=rodou
    else:
        #Reduz o tempo de todos os bloqueados em uma unidade se nao havia ninguem pronto
        tempo+=1
```

Nesta, temos um simulador avançado, com E/S e novos processos chegando em momentos diferentes:

```python

quantum = 2
tempoBloq = 2

escalonador = round_robin([])
bloqueados = []


maximo = 10
chanceNovoProcesso = 60
chanceIo = 30
minTime = 4
maxTime = 10


contaProc = 1
tempo = 0

#descomente essa linha caso queira que os random sempre dêem o mesmo resultado
#random.seed(0)

while tempo<maximo or len(escalonador.prontos)>0:
    
    #Novo processo tem chanceProcesso% de chance surgir enquanto o tempo não chegar no máximo
    if(tempo<maximo and random.randint(1,100)<chanceNovoProcesso):
        p = Processo('P'+str(contaProc),random.randint(1,chanceIo),random.randint(minTime,maxTime),0,tempo)
        print("Processo",p.nome," chegou no tempo",tempo)
        escalonador.pronto(p)
        contaProc+=1
        

    p = escalonador.proximo()
    
    if(p is not None):
        rodou, fezio = p.roda(quantum)
        if(fezio and p.tam>0):
            bloqueados.append([p,tempoBloq+1]) #Adiciona o processo que fez e/s aos bloqueados
        elif(p.tam>0):
            escalonador.pronto(p)
        total-=rodou
        tempo+=rodou
    else:
        #Reduz o tempo de todos os bloqueados em uma unidade
        tempo+=1
    if(len(bloqueados)>0):
        for i in bloqueados:
            i[1]-=1
            if(i[1]==0):
                escalonador.pronto(i[0])
                del i  

### Laboratório: ###

Neste laboratório vocês (em duplas) irão implementar 3 escalonadores:

1 - MLFQ: Adicione parâmetros para definir o número de filas, o quantum de cada uma e o tempo de boost.

2 - Loteria: tem um atributo `prioridade` no Processo, use ele para definir um número aleatório de bilhetes, similar ao que ocorre com e/s e outros parämetros

3 - Stride: Similar ao loteria.



In [33]:
import numpy as np
import random

class Processo(object):
    def __init__(self,pnome,pio,ptam,prioridade,tempoChegada):
        self.nome = pnome
        self.io = pio # Probabilidade de fazer E/S, inicialmente zero
        self.tam = ptam # Quantos Timeslices sao necessarios para terminar
        self.prioridade = prioridade # Prioridade, eh desnecessaria agora 
        self.chegada = 0


    def roda(self,quantum=None): # se rodar sem quantum, o processo roda ate o fim
        if(random.randint(1,100)<self.io): #Verifica se fez E/S
            self.tam-=1
            print(self.nome," fez e/s, falta ",self.tam)
            return 1, True #True que fez E/S
            
            
        if(quantum is None or self.tam<quantum):
            quantum = self.tam
        self.tam -=quantum
        print(self.nome," rodou por ",quantum," timeslice, faltam ",self.tam)
        return quantum, False # False se nao fez E/S

In [34]:
class EscalonadorMLFQ(object):
    def __init__(self, n_filas, quantums, tempo_boost):
        self.n_filas = n_filas
        self.quantums = quantums
        self.tempo_boost = tempo_boost
        self.filas = [[] for _ in range(n_filas)]
        self.tempo_total = 0

    def pronto(self, processo):
        processo.nivel = getattr(processo, 'nivel', 0)
        self.filas[processo.nivel].append(processo)

    def proximo(self):
        self.tempo_total += 1
        if self.tempo_total % self.tempo_boost == 0:
            self._boost()

        for i in range(self.n_filas):
            if self.filas[i]:
                p = self.filas[i].pop(0)
                p.quantum = self.quantums[i]
                return p
        return None

    def _boost(self):
        todos_procs = []
        for fila in self.filas:
            todos_procs.extend(fila)
            fila.clear()
        for p in todos_procs:
            p.nivel = 0
            self.filas[0].append(p)


In [35]:
class EscalonadorLoteria(object):
    def __init__(self, procs=[]):
        self.procs = procs.copy()

    def pronto(self, processo):
        self.procs.append(processo)

    def proximo(self):
        if not self.procs:
            return None

        total_tickets = sum(p.prioridade for p in self.procs)
        sorteado = random.randint(1, total_tickets)
        acumulado = 0

        for p in self.procs:
            acumulado += p.prioridade
            if acumulado >= sorteado:
                self.procs.remove(p)
                return p


In [36]:
class EscalonadorStride(object):
    def __init__(self):
        self.procs = []

    def pronto(self, processo):
        processo.stride = getattr(processo, 'stride', 0)
        processo.pass_value = getattr(processo, 'pass_value', 0)
        processo.ticket = processo.prioridade
        processo.stride = 10000 // processo.ticket
        self.procs.append(processo)

    def proximo(self):
        if not self.procs:
            return None
        self.procs.sort(key=lambda p: p.pass_value)
        p = self.procs.pop(0)
        p.pass_value += p.stride
        return p


In [ ]:
def simulador_basico(escalonador, procs, total, tempo_limite, quantum=2, tempo_bloq=1, seed=):
    bloqueados = []
    tempo = 0

    if seed is not None:
        random.seed(seed)

    # Colocando todos os processos na fila de prontos
    for p in procs:
        escalonador.pronto(p)

    while total > 0 and tempo < tempo_limite:  # Adicionando a condição de tempo limite
        p = escalonador.proximo()
        if p is not None:
            rodou, _ = p.roda(quantum)  # Rodando o processo com o quantum
            if p.tam > 0:
                escalonador.pronto(p)
            total -= rodou
            tempo += rodou
        else:
            # Reduz o tempo de todos os bloqueados (se houvesse)
            tempo += 1

    if tempo >= tempo_limite:
        print("Limite de tempo alcançado!")
    else:
        print("Simulação finalizada em", tempo, "unidades de tempo.")
    
    return tempo

In [38]:
def simulador_avancado(escalonador, quantum=2, tempo_bloq=2, maximo=10, chance_novo_processo=60, chance_io=30, min_time=4, max_time=10, seed=None):
    bloqueados = []
    conta_proc = 1
    tempo = 0
    total = 0

    if seed is not None:
        random.seed(seed)

    while tempo < maximo or len(escalonador.prontos) > 0 or len(bloqueados) > 0:
        # Novo processo com chance_novo_processo%
        if tempo < maximo and random.randint(1, 100) < chance_novo_processo:
            p = Processo(
                'P' + str(conta_proc),
                random.randint(1, chance_io),
                random.randint(min_time, max_time),
                0,
                tempo
            )
            print(f"Processo {p.nome} chegou no tempo {tempo}")
            escalonador.pronto(p)
            conta_proc += 1
            total += p.tam

        p = escalonador.proximo()

        if p is not None:
            rodou, fez_io = p.roda(quantum)
            if fez_io and p.tam > 0:
                bloqueados.append([p, tempo_bloq + 1])
            elif p.tam > 0:
                escalonador.pronto(p)
            total -= rodou
            tempo += rodou
        else:
            tempo += 1  # tempo passa se ninguém pronto

        # Atualiza bloqueados
        desbloquear = []
        for b in bloqueados:
            b[1] -= 1
            if b[1] == 0:
                escalonador.pronto(b[0])
                desbloquear.append(b)
        for b in desbloquear:
            bloqueados.remove(b)

    print("Simulação finalizada em", tempo, "unidades de tempo.")
    return tempo


## Responda ##

#### Sobre Loteria: ####

1 - Rode a simulação mais simples com apenas dois processos, um com 100 tickets e outro com 1, com tempo 100 para terminar. O que aconteceu? O segundo processo conseguiu rodar alguma vez? Testando com outras sementes o resultado se manteve?

2 - Rode a simulação simples com dois processos com 100 tickets e 100 timeslices. Calcule a Unfairness para quantum=2. Repita a operação para quantum 10, 20, 50 e 100. O que aconteceu?

3 - Rode novamente dois processos com 100 tickets no simulador simples e quantum=2. Calcule a unfairness conforme o tamanho dos processos aumenta e faça um gráfico similar ao dos slides.

4 - Rode o simulador maior com três tipos de processo possíveis: um com 10 tickets, um com 20 e um com 50. O que aconteceu com os que receberam menos tickets? Eles tiveram chance de rodar?

In [49]:
# Definição dos processos
nprocs = 2
nomes = ['A','B']
tickets = [100,1]
tamanho = np.array([100,100])
chanceio = [0,0]

total = tamanho.sum()

procs = []
for i in range(nprocs):
    p = Processo(nomes[i], chanceio[i], tamanho[i], tickets[i], 0)
    procs.append(p)

# Simulação com Loteria
escalonador = EscalonadorLoteria(procs)
simulador_basico(escalonador, procs, total, tempo_limite=100)

A  rodou por  2  timeslice, faltam  98
A  rodou por  2  timeslice, faltam  96
A  rodou por  2  timeslice, faltam  94
A  rodou por  2  timeslice, faltam  92
A  rodou por  2  timeslice, faltam  90
A  rodou por  2  timeslice, faltam  88
A  rodou por  2  timeslice, faltam  86
A  rodou por  2  timeslice, faltam  84
A  rodou por  2  timeslice, faltam  82
A  rodou por  2  timeslice, faltam  80
A  rodou por  2  timeslice, faltam  78
A  rodou por  2  timeslice, faltam  76
A  rodou por  2  timeslice, faltam  74
A  rodou por  2  timeslice, faltam  72
A  rodou por  2  timeslice, faltam  70
A  rodou por  2  timeslice, faltam  68
A  rodou por  2  timeslice, faltam  66
A  rodou por  2  timeslice, faltam  64
A  rodou por  2  timeslice, faltam  62
A  rodou por  2  timeslice, faltam  60
A  rodou por  2  timeslice, faltam  58
A  rodou por  2  timeslice, faltam  56
A  rodou por  2  timeslice, faltam  54
A  rodou por  2  timeslice, faltam  52
A  rodou por  2  timeslice, faltam  50
A  rodou por  2  timeslic

100

R: Como o processo A tinha 100 tickets e B apenas 1, acabou que B teve uma competição desvantajosa, ainda mais com a linha de semente aleatória comentada. Se random.seed for descomentada, o processo B apenas rodará ao final, ainda mais com semente = 0. 

#### Sobre Stride ####

1 - Repita o cenário da 2 de loteria e faça um novo gráfico.

2 - No simulador maior, qual o efeito que ter muitos processos fazendo E/S tem no algoritmo? O que pode ser feito com a posição do processo quando ele volta de E/S?

3 - No simulador maior, rode um cenário igual para stride e loteria (com a mesma semente aleatória) e calcule tempo de resposta, de execução e __tempo de espera__ (tempo que o processo passou pronto). O que mudou entre os dois?

#### Sobre MLFQ ####

1 - Rode o MLFQ no segundo simulador com duas filas e sem boost. O que aconteceu? 

2 - Quais configurações de parâmetros transformam o MLFQ no Round Robin?

3 - Rodando no primeiro simulador, faça 1 dos quatro processos ser 100\% CPU-Bound e os outros três 60\% I/O Bound. Use a configuração com 3 filas, com 5, 3 e 1 timeslices para as fila em ordem de priorida. De quanto tempo deve ser o Boost para que o processo CPU-Bound não sofra starvation e tenha CPU 1 vez a cada 20 timeslices?

4 - Teste diversas configurações de número de filas, quantum das filas e boost no simulador maior e ache a que leva ao melhor tempo de execução.